# tubelet-service gRPC client reference

### Initialization

In [28]:
import grpc

from tubelet_service_pb2_grpc import tubelet_serviceStub
from tubelet_service_pb2 import Empty

channel = grpc.insecure_channel('localhost:50051')
tubc = tubelet_serviceStub(channel)

### get_version()

In [27]:
response = tubc.get_version(request=Empty())
print("Version:",response.version)

Version: 1.1.1


### add_observation2d()

In [40]:
from tubelet_service_pb2 import add_observation2d_request, point2d, observation

response = tubc.add_observation2d(
    request=add_observation2d_request(
        pos = point2d(x=23,y=41),
        obs = observation(class_index=10,prob=0.84)
    )
)

### get_all_tubelets2d()

In [62]:
response = tubc.get_all_tubelets2d(request=Empty())
index = 0
for t in response.tubelets:    
    print(index, t)
    index+=1

0 pos {
  x: 1.0
  y: 1.0
}
observations {
}
observations {
  class_index: 1
}
observations {
  class_index: 2
}
observations {
  class_index: 3
}
observations {
  class_index: 4
}

1 pos {
  x: 2.0
  y: 3.0
}
observations {
}
observations {
  class_index: 1
  prob: 0.01
}
observations {
  class_index: 2
  prob: 0.02
}
observations {
  class_index: 3
  prob: 0.03
}
observations {
  class_index: 4
  prob: 0.04
}

2 pos {
  x: 3.0
  y: 5.0
}
observations {
}
observations {
  class_index: 1
  prob: 0.02
}
observations {
  class_index: 2
  prob: 0.04
}
observations {
  class_index: 3
  prob: 0.06
}
observations {
  class_index: 4
  prob: 0.08
}

3 pos {
  x: 4.0
  y: 7.0
}
observations {
}
observations {
  class_index: 1
  prob: 0.03
}
observations {
  class_index: 2
  prob: 0.06
}
observations {
  class_index: 3
  prob: 0.09
}
observations {
  class_index: 4
  prob: 0.12
}

4 pos {
  x: 5.0
  y: 9.0
}
observations {
}
observations {
  class_index: 1
  prob: 0.04
}
observations {
  class_i

### API deinitialization

In [5]:
channel.close()